In [1]:
import h2o
import random
import mlflow
import mlflow.h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
h2o.init()

wine = h2o.import_file(path="wine-quality.csv")
r = wine['quality'].runif()
train = wine[r  < 0.7]
test  = wine[0.3 <= r]

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,53 mins 41 secs
H2O cluster timezone:,Europe/Brussels
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,4 months and 14 days !!!
H2O cluster name:,H2O_from_python_toon_2s41u5
H2O cluster total nodes:,1
H2O cluster free memory:,3.204 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [2]:
def trainRandomForest(ntrees):
    with mlflow.start_run():
        rf = H2ORandomForestEstimator(ntrees=ntrees)
        train_cols = [n for n in wine.col_names if n != "quality"]
        rf.train(train_cols, "quality", training_frame=train, validation_frame=test)
        
        mlflow.log_param("ntrees", ntrees)
        
        mlflow.log_metric("rmse", rf.rmse())
        mlflow.log_metric("r2", rf.r2())
        mlflow.log_metric("mae", rf.mae())
        
        mlflow.h2o.log_model(rf, "model")

In [3]:
for ntrees in [10, 20, 50, 100, 200]:
    trainRandomForest(ntrees)

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


In [4]:
import yaml

In [5]:
yaml.safe_dump

<function yaml.safe_dump(data, stream=None, **kwds)>